In [4]:
%cd fairseq/

/home/khanh/Projects/KhoaLuan/fairseq


In [5]:
pwd

'/home/khanh/Projects/KhoaLuan/fairseq'

In [ ]:
# #  chuẩn hóa giá trị đầu vào cho dataset 
# import os
# import torchaudio
# import torch
# import glob
# from tqdm import tqdm

# # --- CẤU HÌNH ---
# # Thư mục chứa file wav gốc (Tiếng Việt)
# folder = ['source' , 'target']
# file = ['train' , 'test',  'valid']

# TARGET_SAMPLE_RATE = 16000
# # ----------------

# def convert_to_16k(input_path, output_path):
#     try:
#         # Load audio
#         waveform, sample_rate = torchaudio.load(input_path)

#         # 1. Chuyển về Mono (nếu đang là Stereo)
#         if waveform.shape[0] > 1:
#             waveform = torch.mean(waveform, dim=0, keepdim=True)

#         # 2. Resample về 16000Hz (nếu chưa đúng)
#         if sample_rate != TARGET_SAMPLE_RATE:
#             resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=TARGET_SAMPLE_RATE)
#             waveform = resampler(waveform)

#         # Tạo thư mục cha nếu chưa có
#         os.makedirs(os.path.dirname(output_path), exist_ok=True)

#         # Lưu file
#         torchaudio.save(output_path, waveform, TARGET_SAMPLE_RATE)
        
#     except Exception as e:
#         print(f"Lỗi file {input_path}: {e}")

# def main():
#     # Tìm tất cả file wav trong thư mục input (bao gồm cả thư mục con)
#     for i in folder:
#         for   j in file:
#             INPUT_DIR = f"/home/khanh/Projects/KhoaLuan/my_data/{i}_audio/{j}/" 

#             # Thư mục lưu j đã chuẩn hóa (Sẽ tự tạo nếu chưa có)
#             OUTPUT_DIR = f"/home/khanh/Projects/KhoaLuan/my_data/{i}_wav_16k/{j}/"

#             files = glob.glob(os.path.join(INPUT_DIR, "**/*.wav"), recursive=True)
        
#             print(f"Tìm thấy {len(files)} ({i} in {j}) file audio. Bắt đầu xử lý...")

#             for file_path in tqdm(files):
#                 # Tạo đường dẫn output tương ứng
#                 relative_path = os.path.relpath(file_path, INPUT_DIR)
#                 output_path = os.path.join(OUTPUT_DIR, relative_path)
                
#                 convert_to_16k(file_path, output_path)

#             print("✅ Đã hoàn tất chuẩn hóa dữ liệu về 16kHz!")
#             print(f"Dữ liệu mới nằm tại: {OUTPUT_DIR}")

# if __name__ == "__main__":
#     main()

tair mhubert để trích xuất đặc chưng


In [6]:
# Ví dụ cấu trúc thư mục của bạn: /data/en-vi/train/wav_en/ và /data/en-vi/train/wav_vi/
# train 
!python examples/wav2vec/wav2vec_manifest.py ../data/source/train --dest ../manifest_temp/train_en --ext wav --valid-percent 0
!python examples/wav2vec/wav2vec_manifest.py ../data/target/train --dest ../manifest_temp/train_vn --ext wav --valid-percent 0
# # valid  
!python examples/wav2vec/wav2vec_manifest.py ../data/target/valid --dest ../manifest_temp/dev_vn --ext wav --valid-percent 0
!python examples/wav2vec/wav2vec_manifest.py ../data/source/valid  --dest ../manifest_temp/dev_en --ext wav --valid-percent 0

# #  test 
!python examples/wav2vec/wav2vec_manifest.py ../data/source/test --dest ../manifest_temp/test_en --ext wav --valid-percent 0
!python examples/wav2vec/wav2vec_manifest.py ../data/target/test --dest ../manifest_temp/test_vn --ext wav --valid-percent 0





In [ ]:
%cd ..

xoa file thừa

In [7]:
!mkdir -p  ../manifest_temp/source
# source
!cp ../manifest_temp/dev_en/train.tsv ../manifest_temp/source/valid.tsv

!cp ../manifest_temp/train_en/train.tsv ../manifest_temp/source/train.tsv
!cp ../manifest_temp/test_en/train.tsv ../manifest_temp/source/test.tsv
# target
!mkdir -p ../manifest_temp/target
!cp ../manifest_temp/dev_vn/train.tsv ../manifest_temp/target/valid.tsv
!cp ../manifest_temp/train_vn/train.tsv ../manifest_temp/target/train.tsv
!cp ../manifest_temp/test_vn//train.tsv ../manifest_temp/target/test.tsv

In [9]:
# xoa thu muc cu di 
!rm -r ../manifest_temp/dev_vn/
!rm -r ../manifest_temp/dev_en/
!rm -r ../manifest_temp/train_en/
!rm -r ../manifest_temp/train_vn/
!rm -r ../manifest_temp/test_en/
!rm -r ../manifest_temp/test_vn/

In [10]:
# Tạo thư mục chứa feature tạm thời
!mkdir -p /home/khanh/Projects/KhoaLuan/hubert_feats

# Chạy lệnh dump (Lưu ý thay HUBERT_PATH đúng chỗ bạn vừa tải)
!python examples/hubert/simple_kmeans/dump_hubert_feature.py \
    /home/khanh/Projects/KhoaLuan/manifest_temp/target/ \
    train \
    /home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt \
    11 \
    1 \
    0 \
    /home/khanh/Projects/KhoaLuan/hubert_feats \
    --max_chunk 1600000

2026-01-24 09:47:04 | INFO | dump_hubert_feature | Namespace(tsv_dir='/home/khanh/Projects/KhoaLuan/manifest_temp/target/', split='train', ckpt_path='/home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt', layer=11, nshard=1, rank=0, feat_dir='/home/khanh/Projects/KhoaLuan/hubert_feats', max_chunk=1600000)
2026-01-24 09:47:07 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/khanh/Projects/KhoaLuan/fairseq
2026-01-24 09:47:07 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/home/khanh/Projects/KhoaLuan/checkpoints/', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/wnhsu/experiments/hubert/kmeans/mhubert_vp_en_es_fr_it2_400k/en_es_fr.layer9.km500', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio'

In [11]:
!python examples/hubert/simple_kmeans/learn_kmeans.py \
    /home/khanh/Projects/KhoaLuan/hubert_feats \
    train \
    1 \
    /home/khanh/Projects/KhoaLuan/kmeans_model.joblib \
    1000 \
    --percent 0.3

2026-01-24 09:54:35 | INFO | root | Namespace(feat_dir='/home/khanh/Projects/KhoaLuan/hubert_feats', split='train', nshard=1, km_path='/home/khanh/Projects/KhoaLuan/kmeans_model.joblib', n_clusters=1000, seed=0, percent=0.3, init='k-means++', max_iter=100, batch_size=10000, tol=0.0, max_no_improvement=100, n_init=20, reassignment_ratio=0.0)
2026-01-24 09:54:36 | INFO | learn_kmeans | sampled 463 utterances, 140779 frames from shard 0/1
2026-01-24 09:54:36 | INFO | root | loaded feature with dimension (140779, 768)
Init 1/20 with method k-means++
Inertia for init 1/20: 872291.125
Init 2/20 with method k-means++
Inertia for init 2/20: 868583.75
Init 3/20 with method k-means++
Inertia for init 3/20: 869889.3125
Init 4/20 with method k-means++
Inertia for init 4/20: 871182.3125
Init 5/20 with method k-means++
Inertia for init 5/20: 865968.0
Init 6/20 with method k-means++
Inertia for init 6/20: 873047.5625
Init 7/20 with method k-means++
Inertia for init 7/20: 866519.875
Init 8/20 with met

Trích xuất đặc trưng (Feature Extraction): Bạn cần dump các vector đặc trưng từ mHuBERT ra để train K-means.

In [ ]:
# Lưu đoạn này vào file make_dict.py rồi chạy: python make_dict.py
# Hoặc chạy trực tiếp trong terminal python

path = "/home/khanh/Projects/KhoaLuan/checkpoints/dict.txt"

with open(path, "w") as f:
    # Tạo 1003 dòng giả
    for i in range(1000):
        f.write(f"token_{i} 1\n")

print(f"Đã tạo xong dict.txt với 1003 dòng tại {path}")

trainign k_means 

In [12]:
!export PYTHONPATH=$PYTHONPATH:$(pwd)

In [13]:
#  cho tâp train 
!PYTHONPATH=. python examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py --feature_type hubert --kmeans_model_path /home/khanh/Projects/KhoaLuan/kmeans_model.joblib --acoustic_model_path /home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt --layer 11 --manifest_path /home/khanh/Projects/KhoaLuan/manifest_temp/target/train.tsv --out_quantized_file_path /home/khanh/Projects/KhoaLuan/train.unit --extension .wav

2026-01-24 10:17:23 | INFO | __main__ | Namespace(feature_type='hubert', acoustic_model_path='/home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt', layer=11, kmeans_model_path='/home/khanh/Projects/KhoaLuan/kmeans_model.joblib', features_path=None, manifest_path='/home/khanh/Projects/KhoaLuan/manifest_temp/target/train.tsv', out_quantized_file_path='/home/khanh/Projects/KhoaLuan/train.unit', extension='.wav', channel_id=None, hide_fname=False)
2026-01-24 10:17:23 | INFO | __main__ | Extracting hubert acoustic features...
2026-01-24 10:17:24 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/khanh/Projects/KhoaLuan/fairseq
2026-01-24 10:17:24 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/annl/s2st/data/voxpopuli/mHuBERT/en_es_fr', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/wnhsu/experiments/hubert/kmeans/mhubert_vp_en_es_fr_it2_400k/en_es_fr.la

In [14]:
#  cho tập valid 
!PYTHONPATH=. python examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py --feature_type hubert --kmeans_model_path /home/khanh/Projects/KhoaLuan/kmeans_model.joblib --acoustic_model_path /home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt --layer 11 --manifest_path /home/khanh/Projects/KhoaLuan/manifest_temp/target/valid.tsv --out_quantized_file_path /home/khanh/Projects/KhoaLuan/valid.unit --extension .wav

2026-01-24 10:19:41 | INFO | __main__ | Namespace(feature_type='hubert', acoustic_model_path='/home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt', layer=11, kmeans_model_path='/home/khanh/Projects/KhoaLuan/kmeans_model.joblib', features_path=None, manifest_path='/home/khanh/Projects/KhoaLuan/manifest_temp/target/valid.tsv', out_quantized_file_path='/home/khanh/Projects/KhoaLuan/valid.unit', extension='.wav', channel_id=None, hide_fname=False)
2026-01-24 10:19:41 | INFO | __main__ | Extracting hubert acoustic features...
2026-01-24 10:19:44 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/khanh/Projects/KhoaLuan/fairseq
2026-01-24 10:19:44 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/annl/s2st/data/voxpopuli/mHuBERT/en_es_fr', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/wnhsu/experiments/hubert/kmeans/mhubert_vp_en_es_fr_it2_400k/en_es_fr.la

In [15]:
# Copy file .unit (file này có chứa ID|...) sang thành .txt
!mkdir -p ../manifest_temp/train
!mkdir -p ../manifest_temp/valid
!cp /home/khanh/Projects/KhoaLuan/train.unit /home/khanh/Projects/KhoaLuan/manifest_temp/train/train.txt

# Nếu bạn có cả valid thì copy luôn
!cp /home/khanh/Projects/KhoaLuan/valid.unit /home/khanh/Projects/KhoaLuan/manifest_temp/valid/valid.txt

In [16]:
pwd 

'/home/khanh/Projects/KhoaLuan/fairseq'

các điều kiện bắt buộc để chạy chương trình này đó là 
1 : file train và vali phỉa là dạng .txt (chuyển từ unit -> txt)
2: chuyển về đúng đinh dạng .wav nữa (chạy lệnh sed -i 's/|/.wav|/' /home/khanh/Projects/KhoaLuan/manifest_temp/train_vn/train.txt)

In [17]:
#  chuyển train.tsv vào trong train 
!cp ../manifest_temp/source/train.tsv ../manifest_temp/train/train.tsv
!cp ../manifest_temp/source/valid.tsv ../manifest_temp/valid/valid.tsv


In [ ]:
#  sắp xếp lại 

In [23]:
# Set đường dẫn gốc cho gọn

!PYTHONPATH=. python examples/speech_to_speech/preprocessing/prep_s2ut_data.py --source-dir /home/khanh/Projects/KhoaLuan/data/source --target-dir /home/khanh/Projects/KhoaLuan/manifest_temp/train --data-split train  --output-root /home/khanh/Projects/KhoaLuan/data_bin/ --reduce-unit 

Generating manifest...
Processing train
100%|█████████████████████████████████████| 1541/1541 [00:00<00:00, 5596.54it/s]
Processed 1541 samples
Writing manifest to /home/khanh/Projects/KhoaLuan/data_bin/train.tsv...


In [24]:
!PYTHONPATH=. python examples/speech_to_speech/preprocessing/prep_s2ut_data.py --source-dir /home/khanh/Projects/KhoaLuan/data/source --target-dir /home/khanh/Projects/KhoaLuan/manifest_temp/valid --data-split valid  --output-root /home/khanh/Projects/KhoaLuan/data_bin/ --reduce-unit 

Generating manifest...
Processing valid
100%|███████████████████████████████████████| 192/192 [00:00<00:00, 5358.39it/s]
Processed 192 samples
Writing manifest to /home/khanh/Projects/KhoaLuan/data_bin/valid.tsv...


In [28]:
import os

# Đường dẫn file dict (sửa lại nếu cần)
output_file = '/home/khanh/Projects/KhoaLuan/data_bin/dict.txt'

print(f"Đang tạo {output_file}...")

with open(output_file, 'w') as f:
    # mHuBERT dùng 1000 clusters -> chạy từ 0 đến 999
    for i in range(1000):
        # Cấu trúc chuẩn: <Tên Unit> <Khoảng trắng> <Tần suất giả>
        f.write(f"{i} 1\n")

print("✅ Đã tạo xong! File chuẩn phải có 1000 dòng.")

Đang tạo /home/khanh/Projects/KhoaLuan/data_bin/dict.txt...
✅ Đã tạo xong! File chuẩn phải có 1000 dòng.


lệnh train


In [29]:
!fairseq-train /home/khanh/Projects/KhoaLuan/data_bin \
    --config-yaml config.yaml \
    --task speech_to_text \
    --arch s2t_transformer_s \
    --share-decoder-input-output-embed \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --adam-eps 1e-8 \
    --lr 1e-3 --lr-scheduler inverse_sqrt \
    --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --max-tokens 2000 \
    --update-freq 8 \
    --max-update 3000 \
    --log-format json --log-interval 100 \
    --save-dir /home/khanh/Projects/KhoaLuan/checkpoints/s2ut_official \
    --keep-last-epochs 10 \
    --keep-best-checkpoints 5 \
    --patience 20 \
    --num-workers 4 \
    --fp16

2026-01-24 10:55:19 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [32]:
!fairseq-generate /home/khanh/Projects/KhoaLuan/data_bin \
    --config-yaml config.yaml \
    --path ../checkpoints/s2ut_official/checkpoint_best.pt \
    --task speech_to_text \
    --gen-subset valid \
    --beam 5 \
    --batch-size 1 \
    --results-path /home/khanh/Projects/KhoaLuan/results/test_run

2026-01-24 13:54:15 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

In [ ]:
#  tesst vocoder 
# Tạo thư mục kết quả
!mkdir -p results_audio


# Chạy lệnh convert
!python examples/speech_to_speech/generate_waveform_from_code.py \
    --in-code-file ../test_unit.txt \
    --vocoder ../vocoder/g_00500000 \
    --vocoder-cfg ../vocoder/config.json \
    --results-path ../results_audio \